In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("datasets/movies.csv")

In [4]:
import re

In [5]:
x = re.search("\(\d+\)",df.iloc[1,1])

In [6]:
df.iloc[1,1][x.start()+1:x.end()-1]

'1995'

In [17]:
bson = []
for i in range(len(df)):
    dct = {}
    dct['movieId'] = str(df.iloc[i,0])
    dct['title'] = df.iloc[i,1]
    x = re.search("\(\d+\)",df.iloc[i,1])
    if x is None:
        dct['year'] = '1979'
    else:
        dct['year'] = df.iloc[i,1][x.start()+1:x.end()-1]
    dct['genres'] = df.iloc[i,2].split('|')
    bson.append(dct)

In [18]:
import json

In [19]:
with open('datasets/data.json', 'w') as f:
    json.dump(bson, f)

# Recommendation

In [1]:
import pandas as pd
import numpy as np

In [2]:
metadata = pd.read_csv("datasets/movies_metadata.csv", low_memory=False)

In [3]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [4]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [5]:
len(metadata['overview'])

45466

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf = TfidfVectorizer(stop_words='english', dtype=np.float32)

In [8]:
metadata['overview'] = metadata['overview'].fillna('')

In [9]:
tfidf_matrix = tfidf.fit_transform(metadata['overview']).astype(np.float16, copy=False)

In [10]:
tfidf_matrix

In [11]:
tfidf_matrix.shape

In [12]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [13]:
cosine_sim[0].toarray()

In [38]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [40]:
def get_recommendations(title, cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = cosine_sim[idx].toarray().squeeze()
    # Get sorted indices
    movie_indices = sim_scores.argsort()[::-1][1:11]
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [76]:
get_recommendations('The Dark Knight Rises', cosine_sim)

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [77]:
get_recommendations('The Godfather', cosine_sim)

1178               The Godfather: Part II
44030    The Godfather Trilogy: 1972-1990
1914              The Godfather: Part III
23126                          Blood Ties
11297                    Household Saints
34717                   Start Liquidation
10821                            Election
38030            A Mother Should Be Loved
17729                   Short Sharp Shock
26293                  Beck 28 - Familjen
Name: title, dtype: object

# using credits and keywords

In [14]:
credits = pd.read_csv('datasets/credits.csv')
keywords = pd.read_csv('datasets/keywords.csv')

In [15]:
metadata = metadata.drop([19730, 29503, 35587])

In [16]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

In [17]:
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [18]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [19]:
from ast import literal_eval

In [20]:
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [21]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [22]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [23]:
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [24]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [25]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [26]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [27]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [28]:
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [29]:
metadata[['soup']].head(3)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...
2,fishing bestfriend duringcreditsstinger walter...


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
count = CountVectorizer(stop_words='english', dtype=np.float32)

In [32]:
count_matrix = count.fit_transform(metadata['soup'])

In [33]:
count_matrix.shape

(46628, 73881)

In [35]:
cosine_sim2 = count_matrix.dot(count_matrix.T)

In [41]:
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [42]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

12589         The Dark Knight
10210           Batman Begins
7772                 Mitchell
516         Romeo Is Bleeding
24090               Quicksand
10853          Helter Skelter
11463            The Prestige
9311                   Shiner
45156    Payback: Straight Up
25038                Deadfall
Name: title, dtype: object

In [43]:
get_recommendations('The Godfather', cosine_sim2)

1934     The Godfather: Part III
7772                    Mitchell
1199      The Godfather: Part II
3635            The Conversation
28683     In the Name of the Law
4012            Gardens of Stone
10540          Two for the Money
13474             Chinese Coffee
33611            A Bigger Splash
1856           On the Waterfront
Name: title, dtype: object